In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [24]:
# df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)

In [25]:
# df.head()

In [26]:
# df.shape

In [27]:
# df_2 = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
# df_2.head()

In [28]:
# df_2.shape

In [29]:
# df_3 = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)
# df_3.head()

In [30]:
# df_3.shape

In [32]:
import os, glob

In [33]:
files = glob.glob('*.txt')

In [34]:
files

['amazon_cells_labelled.txt', 'imdb_labelled.txt', 'yelp_labelled.txt']

In [39]:
data = ""
table = str.maketrans('', '', string.punctuation)
for i in range(len(files)):
    file = open(files[i])
    text = file.read()
    text = text.translate(table)
    data += (text)
    file.close()

In [42]:
file = open('all_reviews.txt','w')
file.write(data)
file.close()

In [43]:
new_df = pd.read_csv('all_reviews.txt', sep='\t', header=None)

In [11]:
# new_df = new_df.append(df)
# new_df = new_df.append(df_2)
# new_df = new_df.append(df_3)

In [44]:
new_df.shape

(3000, 2)

In [45]:
new_df.columns = ['Review', 'Sentiment']

In [46]:
new_df.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,Good case Excellent value,1
2,Great for the jawbone,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great,1


In [47]:
def text_processing(documents):
    table = str.maketrans('', '', string.punctuation)
    for i in range(len(documents)):
        documents['Review'].iloc[i] = documents['Review'].iloc[i].lower().translate(table)
        
    tokens = []
    for i in range(len(documents)):
        tokens.append(word_tokenize(documents['Review'].iloc[i]))
        
    eng_stopwords = stopwords.words('english')
    wordsList = []
    for tokenList in tokens:
        words = []
        for token in tokenList:
            if token not in eng_stopwords:
                words.append(token)
        wordsList.append(words)
        
    wnet = WordNetLemmatizer()
    for i in range(len(wordsList)):
        for j in range(len(wordsList[i])):
            wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos='v')
            
    for i in range(len(wordsList)):
        wordsList[i] = ' '.join(wordsList[i])
    
    return wordsList

In [48]:
wordsList = text_processing(new_df)

F:\anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [50]:
wordsList[0:10]

['way plug us unless go converter',
 'good case excellent value',
 'great jawbone',
 'tie charger conversations last 45 minutesmajor problems',
 'mic great',
 'jiggle plug get line right get decent volume',
 'several dozen several hundred contact imagine fun send one one',
 'razr owneryou must',
 'needle say waste money',
 'waste money time']

In [51]:
len(wordsList)

3000

In [52]:
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(wordsList)

In [53]:
vector

<3000x4532 sparse matrix of type '<class 'numpy.float64'>'
	with 18124 stored elements in Compressed Sparse Row format>

In [54]:
words_array = vector.toarray()

In [55]:
words_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [56]:
words_array.shape

(3000, 4532)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(words_array, new_df['Sentiment'], test_size=0.25)

In [67]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [68]:
y_pred = reg.predict(x_test)

In [69]:
accuracy_score(y_test, y_pred)

0.7906666666666666

In [70]:
confusion_matrix(y_test, y_pred)

array([[309,  50],
       [107, 284]], dtype=int64)

In [71]:
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB()

In [72]:
y_pred = nb.predict(x_test)
accuracy_score(y_test, y_pred)

0.804

In [73]:
confusion_matrix(y_test, y_pred)

array([[294,  65],
       [ 82, 309]], dtype=int64)

In [64]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [65]:
y_pred = knn.predict(x_test)
accuracy_score(y_test, y_pred)

0.772

In [66]:
confusion_matrix(y_test, y_pred)

array([[259, 100],
       [ 71, 320]], dtype=int64)

In [79]:
review = "I have been fan of Mortal Kombat for a few years, and this movie is perfect for the videogame, if you know the history and characters you will be excited if you watch it, since the cast, story and fights are exactly what I imagined. Mortal Kombat is cheesy in its way, and I love that they captured that, but some people could not understand this. Anyways, I cannot wait for the second part of this movie."
test_df = pd.DataFrame({'Review' : [review]})

In [80]:
wordsList_test = text_processing(test_df)

In [81]:
wordsList_test

['fan mortal kombat years movie perfect videogame know history character excite watch since cast story fight exactly imagine mortal kombat cheesy way love capture people could understand anyways wait second part movie']

In [82]:
vec_test = tfidf.transform(wordsList_test).toarray()

In [83]:
nb.predict(vec_test)

array([1], dtype=int64)